# Compare DMD models

In [ ]:
from pathlib import Path
from ctf4science.eval_module import extract_metrics_in_order
import os
import yaml

import numpy as np

from matplotlib import cm
import matplotlib.pyplot as plt

def spider_plot(results, baseline_batches, model_batches, dataset_name,
                baseline_colormap = cm.jet_r, model_colormap = cm.plasma):

    categories = [f"E{i+1}" for i in range(10)]
    N = len(categories)

    # Create full batch paths
    all_batches = baseline_batches + model_batches
    
    # Generate colors for each batch
    baseline_colors = [baseline_colormap(i / len(baseline_batches)) for i in range(len(baseline_batches))]
    model_colors = [model_colormap(i / len(model_batches)) for i in range(len(model_batches))]
    colors = baseline_colors + model_colors

    # Create the figure and polar subplot
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection="polar"))

    # Plot each batch
    for i, batch_path in enumerate(all_batches):
        # Retrieve metrics (assuming results uses full paths as keys)
        values = list(results[str(batch_path)].values())
        # Close the plot by repeating the first value
        values += values[:1]
        # Calculate angles for the categories
        angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
        angles += angles[:1]

        # Check values
        values = [v if v > -100 else -100 for v in values]
        
        # Plot the line and fill the area
        ax.plot(angles, values, label=all_batches[i], color=colors[i], alpha=0.7, marker='p', linestyle='--')
        ax.fill(angles, values, color=colors[i], alpha=0.1)
        
    # Set category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    
    # Set y-axis limits (assuming metrics are percentages; adjust if needed)
    ax.set_ylim(-100, 100)
    
    # Add a legend outside the plot
    ax.legend(loc="upper right", bbox_to_anchor=(1.7, 1.1))
    
    # Add a title
    ax.set_title(f"Performance Metrics E1 to E{N} ({dataset_name})", pad=20)
    
    # Display the plot
    plt.show()

## Lorenz
Let us load the results

In [ ]:
# Dataset name
dataset_name = "ODE_Lorenz"

# Dictionary to store results for all batches
results = {}

baseline_batches = [
    "CTF_NaiveBaselines_average/batch_avg_20250425_152911/",
]

model_batches = [
    'classicDMD/'   + os.listdir(f"../../../results/{dataset_name}/classicDMD/")[-1],
    'hankelDMD/'    + os.listdir(f"../../../results/{dataset_name}/hankelDMD/")[-1],
    'highorderDMD/' + os.listdir(f"../../../results/{dataset_name}/highorderDMD/")[-1],
    'bagoptDMD/'    + os.listdir(f"../../../results/{dataset_name}/bagoptDMD/")[-1]
]

all_batches = baseline_batches + model_batches

for batch in all_batches:
    # Construct the path to batch_results.yaml
    batch_dir = Path("../../../results") / dataset_name / batch
    batch_results_path = batch_dir / "batch_results.yaml"
    
    # Load the batch_results.yaml file
    with open(batch_results_path, "r") as f:
        batch_results = yaml.safe_load(f)
    
    # Extract metric values in the correct order
    metric_values = extract_metrics_in_order(dataset_name, batch_results)
    
    # Map values to E1, E2, ..., E12
    e_metrics = {f"E{i+1}": value for i, value in enumerate(metric_values)}
    
    # Store the transformed metrics with the batch path as the key
    results[batch] = e_metrics

spider_plot(results, baseline_batches, model_batches, dataset_name)

## KS
Let us load the results

In [ ]:
# Dataset name
dataset_name = "PDE_KS"

# Dictionary to store results for all batches
results = {}

baseline_batches = [
    "CTF_NaiveBaselines_average/batch_avg_20250425_152911/",
]

model_batches = [
    'classicDMD/'   + os.listdir(f"../../../results/{dataset_name}/classicDMD/")[-1],
    'hankelDMD/'    + os.listdir(f"../../../results/{dataset_name}/hankelDMD/")[-1],
    'highorderDMD/' + os.listdir(f"../../../results/{dataset_name}/highorderDMD/")[-1],
    'bagoptDMD/'    + os.listdir(f"../../../results/{dataset_name}/bagoptDMD/")[-1]
]

all_batches = baseline_batches + model_batches

for batch in all_batches:
    # Construct the path to batch_results.yaml
    batch_dir = Path("../../../results") / dataset_name / batch
    batch_results_path = batch_dir / "batch_results.yaml"
    
    # Load the batch_results.yaml file
    with open(batch_results_path, "r") as f:
        batch_results = yaml.safe_load(f)
    
    # Extract metric values in the correct order
    metric_values = extract_metrics_in_order(dataset_name, batch_results)
    
    # Map values to E1, E2, ..., E12
    e_metrics = {f"E{i+1}": value for i, value in enumerate(metric_values)}
    
    # Store the transformed metrics with the batch path as the key
    results[batch] = e_metrics

spider_plot(results, baseline_batches, model_batches, dataset_name)